In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import requests
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import urllib
import os
from PIL import Image
from io import StringIO
import pandas as pd

In [3]:
s=Service(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.53/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\fujistu\.wdm\drivers\chromedriver\win32\103.0.5060.53]


# Scrapping the data

In [4]:
# Launch the browser
driver = webdriver.Chrome(service=s)
# maximise the window
driver.maximize_window()
# define the url
url = 'https://dermnetnz.org/image-library/'
# open the url in chrome
driver.get(url)

name = []
url = []
img = []
time.sleep(4)

# Find all the names by their class names
name_elements = driver.find_elements(By.CLASS_NAME, 'imageList__group__item__copy')

# find all the urls with their class name
elem = driver.find_elements(By.CLASS_NAME, 'imageList__group__item')

# image source
img_src = driver.find_elements(By.TAG_NAME, 'img')

# extract the names from their elements
for i in name_elements:
    name.append(i.text)
    
# extract the url
for i in elem:
    l = i.get_attribute("href")
    url.append(l)

# get the source url of each image
for i in img_src:
    k = i.get_attribute('src')
    img.append(k)

# removing all the extra images extracted
final_images_src = img[1:len(name)+1]

# create a new directory in the current directory if it does not exist
# os.mkdir('images') 

# Download the images
for i in range(len(final_images_src)):
    r1 = requests.get(final_images_src[i]).content
    with open(str(name[i].replace('/', ' ').replace(' ', '_'))+'.jpg','wb') as f:
        f.write(r1)
#     time.sleep(2)
        
        
driver.close()

# Creating the Dataframe

In [5]:
len(name), len(url), len(final_images_src)

(294, 294, 294)

In [6]:
df = pd.DataFrame({'Disease':name,
            'Disease_URL': url,
            'thumbnail_URL' : final_images_src})

In [7]:
df

,Disease,Disease_URL,thumbnail_URL
0,Acne affecting the back images,https://dermnetnz.org/topics/acne-affecting-th...,https://dermnetnz.org/assets/manualthumbnails/...
1,Acne affecting the face images,https://dermnetnz.org/topics/acne-face-images,https://dermnetnz.org/assets/Uploads/Screen-Sh...
2,Acne and other follicular disorder images,https://dermnetnz.org/image-catalogue/acne-and...,https://dermnetnz.org/assets/Uploads/ocular-ro...
3,Acquired dermal macular hyperpigmentation images,https://dermnetnz.org/topics/acquired-dermal-m...,https://dermnetnz.org/assets/Uploads/scaly/lp-...
4,Acral lentiginous melanoma images,https://dermnetnz.org/topics/acral-lentiginous...,https://dermnetnz.org/assets/Uploads/20160516-...
...,...,...,...
289,Vulval lichen sclerosus images,https://dermnetnz.org/topics/vulval-lichen-scl...,https://dermnetnz.org/assets/Uploads/074__Focu...
290,Vulval ulcer images,https://dermnetnz.org/topics/vulval-ulcer-images,https://dermnetnz.org/assets/Uploads/vulval-ul...
291,Vulvovaginal candidiasis images,https://dermnetnz.org/topics/vulvovaginal-cand...,https://dermnetnz.org/assets/Uploads/candida2-...
292,Xanthelasma images,https://dermnetnz.org/topics/xanthelasma-images,https://dermnetnz.org/assets/Uploads/xanthelas...


# Saving the data to csv file

In [8]:
df.to_csv('Diseases`_scrapped.csv')